In [1]:
with open("./input.txt", "r") as f:
    data = f.read().split("\n")
    data = [d for d in data if d]

items = []
import re

for d in data:
    if d.startswith("{") and d.endswith("}"):
        item = {}
        for kv in d[1:-1].split(","):
            k, v = kv.split("=")
            item[k] = int(v)
        items.append([item["x"], item["m"], item["a"], item["s"]])
    else:
        n, w = d[:-1].split("{")
        n = n.upper()

        f = f"def {n}(x,m,a,s):\n"
        for condition in w.split(","):
            regex = r"([xmas])([\<\>])(\d+):(\w+)"

            m = re.match(regex, condition)
            if m:
                k = m.group(1)
                op = m.group(2)
                threshold = m.group(3)
                down = m.group(4)
                f += f"\tif {k} {op} {threshold}:\n"
                f += f"\t\treturn '{down.upper()}'\n"
            else:
                down = condition
                f += f"\treturn '{down.upper()}'"

        exec(f)


def f1(item):
    x, m, a, s = item

    workflow = "IN"
    while workflow != "R" and workflow != "A":
        workflow = eval(f"{workflow}({x},{m},{a},{s})")

    if workflow == "A":
        return x + m + a + s
    return 0


ret = 0
for item in items:
    ret += f1(item)
print(ret)

368523


In [2]:
with open("./input.txt", "r") as f:
    data = f.read().split("\n")
    data = [d for d in data if d]

workflows = {}
import re

for d in data:
    if d.startswith("{") and d.endswith("}"):
        pass
    else:
        n, w = d[:-1].split("{")
        n = n.upper()

        workflow = []
        for condition in w.split(","):
            regex = r"([xmas])([\<\>])(\d+):(\w+)"

            m = re.match(regex, condition)
            if m:
                workflow.append(
                    {
                        "type": "if",
                        "k": m.group(1),
                        "op": m.group(2),
                        "threshold": int(m.group(3)),
                        "down": m.group(4).upper(),
                    }
                )
            else:
                workflow.append({"type": "return", "down": condition.upper()})

        workflows[n] = workflow


def split_ranges(old_ranges, point, cp):
    small = []
    big = []
    if cp == ">":
        for r in old_ranges:
            if point >= r[1]:
                small.append(r)
            elif point < r[0]:
                big.append(r)
            else:
                small.append([r[0], point])
                big.append([point + 1, r[1]])
        return big, small
    elif cp == "<":
        for r in old_ranges:
            if point > r[1]:
                small.append(r)
            elif point <= r[0]:
                big.append(r)
            else:
                small.append([r[0], point - 1])
                big.append([point, r[1]])
        return small, big


# print(split_ranges([[0, 4000]], 4000, ">"))
# print(split_ranges([[0, 4000]], 4000, "<"))
# print(split_ranges([[0, 4000]], 2409, "<"))
# print(split_ranges([[0, 4000]], 2409, ">"))
# print(split_ranges([[0, 2000], [2400, 2500], [2501, 2989]], 2409, ">"))
# print(split_ranges([[0, 2000], [2400, 2500], [2501, 2989]], 2409, "<"))
#
# ([], [[0, 4000]])
# ([[0, 3999]], [[4000, 4000]])
# ([[0, 2408]], [[2409, 4000]])
# ([[2410, 4000]], [[0, 2409]])
# ([[2410, 2500], [2501, 2989]], [[0, 2000], [2400, 2409]])
# ([[0, 2000], [2400, 2408]], [[2409, 2500], [2501, 2989]])


def count_combo(item):
    total = 1
    for k in ["x", "m", "a", "s"]:
        range_length = 0
        for r in item[k]:
            start = r[0]
            end = r[1]
            range_length = end - start + 1
        total *= range_length
    return total


q = []
start = {k: [[1, 4000]] for k in "xmas"}
start["w"] = "IN"
q.append(start)

ret = 0
while q:
    item = q.pop()

    new_items = []

    workflow = workflows[item["w"]]

    for step in workflow:

        if step["type"] == "if":

            match_range, unmatch_range = split_ranges(
                item[step["k"]], step["threshold"], step["op"]
            )

            new_item = dict.copy(item)
            new_item[step["k"]] = match_range
            new_item["w"] = step["down"]
            new_items.append(new_item)

            item[step["k"]] = unmatch_range

        elif step["type"] == "return":
            new_item = dict.copy(item)
            new_item["w"] = step["down"]
            new_items.append(new_item)

    for new_item in new_items:
        if new_item["w"] == "A":
            ret += count_combo(new_item)
        elif new_item["w"] == "R":
            continue
        else:
            q.append(new_item)

print(ret)

124167549767307
